In [426]:
import datetime 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pymongo
import warnings
from dateutil.parser import parse
import datetime as dt
from lxml import etree



class DataScraper:
    
    def __init__(self):
        warnings.filterwarnings("ignore")
        client = pymongo.MongoClient('127.0.0.1', 27017)
        db = client["news"]
        newsCollection = db["news_collection"]

        self.proxies = {
            "https" : "fnyproxy.fnylocal.com:8080",
            "http" : "fnyproxy.fnylocal.com:8080"
        }

        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0'}
        
        
        
    def soup_maker(self, url):
        
        res = requests.get(url, proxies = self.proxies,headers=self.headers, verify = False)
        soup = BeautifulSoup(res.text, "html.parser")
        return soup
            
        
    ####### FOOL ####### //div/div/header/h1[contains(@class, "text-3xl font-medium tracking-tight text-gray-1100 leading-relative-2 md:text-5xl")]
    def scrape_fool_news_links(self):
        
        url = "https://www.fool.com/investing-news/"
        
        soup = self.soup_maker(url)
        
        dom = etree.HTML(str(soup))

        links_element = dom.xpath('//div/div[contains(@class, "flex py-12px text-gray-1100")]//a[contains(@class, "flex-shrink-0 w-1/3 mr-16px sm:w-auto")]')

        new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])

        for link_ele in links_element:
            combine_whole_link = "https://www.fool.com" + link_ele.get('href')
            
            title, text, date = self.go_inside_fool_links(link = combine_whole_link)
            
            new_row = {"headline": title, "alltext": text, "date": date, "source": "FOOL", "url": combine_whole_link}
            new_rows_df = new_rows_df.append(new_row, ignore_index=True)

        return new_rows_df
    
    
    def go_inside_fool_links(self, link):
        # for link in links: 
        soup = self.soup_maker(link)
        dom = etree.HTML(str(soup))

        title_element = dom.xpath('//div/div/header/h1[contains(@class, "text-3xl font-medium tracking-tight text-gray-1100 leading-relative-2 md:text-5xl")]/text()')
        if title_element != []:
            title = title_element[0]
        else:
            title = title_element
        # title = 'titlegelecek'
        
        text_element =dom.xpath('//div/div[contains(@class, "max-w-full")]/div[contains(@class, "article-body")]/p//text()')
        text = ''
        self.text_element = text_element
        for paragraph in text_element:
            text = text + paragraph
        # text= 'textgelecek'

        date_element = dom.xpath('//div/div[contains(@class, "text-lg font-medium text-gray-800 mt-12px md:mt-24px mb-24px")]/text()[2]')
        date = date_element
        try:
            date = date[0]
            date = date[3:-1]
        except:
            pass
        if 'Updated' in date:
            date = date.replace('Updated ', '')
                # orjinal format 'Jul 9, 2024 at 4:51AM'
        # date = pd.to_datetime(date, format='%b %d, %Y at %I:%M%p', errors='coerce')
        
        date = pd.to_datetime(date)
        
        # date = '13'


        return title, text, date
            
        
    ######################
        
    ####### BENZINGA #######
    def urls_benzinga(self):
        
        sections = ["news", "markets", "trading-ideas"]
        urls = []
        for section in sections:
            urls.append("https://www.benzinga.com/" + section)

        return urls
        
    def scrape_benzinga_news_links(self):
        
        urls = self.urls_benzinga()
        new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])


        for url in urls:
            soup = self.soup_maker(url)

            dom = etree.HTML(str(soup))

            links_element = dom.xpath('//div/div[contains(@class, "newsfeed-card text-black")]//a[contains(@class, "post-card-article-link post-card-image image-wrapper")]')


            before_new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])


            for link_ele in links_element:
                combine_whole_link = link_ele.get('href')
                title, text, date = self.go_inside_benzinga_links(link = combine_whole_link)



                new_row = {"headline": title, "alltext": text, "date": date, "source": "BENZINGA", "url": combine_whole_link}
                before_new_rows_df = before_new_rows_df.append(new_row, ignore_index=True)

            new_rows_df = new_rows_df.append(before_new_rows_df, ignore_index=True)

        return new_rows_df
    
    
    def go_inside_benzinga_links(self, link):
        # for link in links:
        soup = self.soup_maker(url = link)
        dom = etree.HTML(str(soup))

        title_element = dom.xpath('//div/div/div[contains(@class, "layout-container mx-auto")]/div[contains(@class, "flex flex-col md:flex-row items-start md:items-center gap-2 md:gap-8")]/h1/text()')
        if title_element != []:
            title = title_element[0]
        else:
            title = title_element
        # title = 'titlegelecekqqq'

        text_element =dom.xpath('//div/div[contains(@itemprop, "articleBody")]/div/div//p//text()')
        text = ''
        self.text_element = text_element
        for paragraph in text_element:
            text = text + paragraph

        # paragraphs = text_element.find_all('p') if text_element else []
        # text = '\n'.join(p.get_text() for p in paragraphs)
        # text= 'textgelecekq1234'

        date_element = dom.xpath('//div/div/div[contains(@class, "article-date-wrap")]/span[contains(@class, "date")]/text()')
        date = date_element[0]
                # orjinal hali: 'July 9, 2024 4:51AM'
        # date = pd.to_datetime(date, format='%B %d, %Y %I:%M%p', errors='coerce')
        
        date = pd.to_datetime(date)
        # date = '13'




        return title, text, date
    
    ######################
    

    
    
    ####### FINANCE-YAHOO #######
    def urls_yahoo(self):
        
        sections = ["latest-news", "stock-market-news/", "yahoo-finance-originals/", "economic-news/", "earnings/", "tech/", "crypto/"]
        urls = []
        for section in sections:
            urls.append("https://finance.yahoo.com/topic/" + section)

        return urls
        
    def scrape_yahoo_news_links(self):
        
        urls = self.urls_yahoo()
        new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])


        for url in urls:
            soup = self.soup_maker(url)

            dom = etree.HTML(str(soup))

            links_element = dom.xpath('//div/div//li[contains(@class, "js-stream-content")]/div[contains(@class, "Py(14px)")]//a')


            before_new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])


            for link_ele in links_element:
                combine_whole_link = link_ele.get('href')
                if combine_whole_link[0] == 'h':
                    title, text, date = self.go_inside_yahoo_links(link = combine_whole_link)
                else:
                    pass



                new_row = {"headline": title, "alltext": text, "date": date, "source": "FINANCE-YAHOO", "url": combine_whole_link}
                before_new_rows_df = before_new_rows_df.append(new_row, ignore_index=True)

            new_rows_df = new_rows_df.append(before_new_rows_df, ignore_index=True)

        return new_rows_df
    
    
    def go_inside_yahoo_links(self, link):
        # for link in links:
        soup = self.soup_maker(url = link)
        dom = etree.HTML(str(soup))

        title_element = dom.xpath('//div//div[contains(@class, "caas-title-wrapper")]//text()')
        if title_element != []:
            title = title_element[0]
        else:
            title = title_element
        # title = 'titlegelecekqqq'

        text_element =dom.xpath('//div/div[contains(@class, "caas-body")]//p//text()')
        text = ''
        for paragraph in text_element:
            text = text + paragraph

        # paragraphs = text_element.find_all('p') if text_element else []
        # text = '\n'.join(p.get_text() for p in paragraphs)
        # text= 'textgelecekq1234'

        date_element = dom.xpath('//div/div[contains(@class, "caas-attr-time-style")]//text()')
        date = date_element[0]
        if 'Updated' in date:
            date_element = dom.xpath('//div/div[contains(@class, "caas-attr-time-style")]//time/text()')
            date = date_element[0]
            
            #orjinal hali: 'Tue, Jul 9, 2024, 4:51AM'
        # date = pd.to_datetime(date, format='%a, %b %d, %Y, %I:%M%p', errors='coerce')
        
        date = pd.to_datetime(date)
        
        # date = date.split(', ', 1)[1].strip()
        
        # date = '13'




        return title, text, date
    
    ######################
    
    
    
    
    

    
    ####### BARRONS ####### need subscription !!!!!!!!!!!!
    def urls_barrons(self):
        
        sections = ["markets", "europe", "asia", "technology", "earnings/", "economy-and-policy"]
        urls = []
        for section in sections:
            urls.append("https://www.barrons.com/topics/" + section)

        return urls
        
    def scrape_barrons_news_links(self):
        
        urls = self.urls_barrons()
        new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])


        for url in urls:
            soup = self.soup_maker(url)

            dom = etree.HTML(str(soup))

            links_element = dom.xpath('//div/div[contains(@class, "BarronsTheme--button-label--3EhEhI0I")]/a')


            before_new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])


            for link_ele in links_element:
                combine_whole_link = link_ele.get('href')
                
                title, text, date = self.go_inside_barrons_links(link = combine_whole_link)




                new_row = {"headline": title, "alltext": "text", "date": "date", "source": "BARRONS", "url": combine_whole_link}
                before_new_rows_df = before_new_rows_df.append(new_row, ignore_index=True)

            new_rows_df = new_rows_df.append(before_new_rows_df, ignore_index=True)

        return new_rows_df
    
    
    def go_inside_barrons_links(self, link):
        # for link in links:
        soup = self.soup_maker(url = link)
        dom = etree.HTML(str(soup))

        # title_element = dom.xpath('//div/div[contains(@class, "standard__HeadlineWrapper-sc-14sjre0-12")]//text()')
        # title = title_element[0]

        # title = 'titlegelecekqqq'
        title = soup.prettify()
        
        
        
        # text_element =dom.xpath('//div/section[contains(@class, "css-yfonvn-Container ef4qpkp0")]//p//text()')
        # text = ''
        # for paragraph in text_element:
        #     text = text + paragraph

        text= 'textgelecekq1234'

        # date_element = dom.xpath('//div/div/p[contains(@class, "css-1v6wp4h-TimeTag")]//text()')
        # date = date_element[0]
        date = '13'




        return title, text, date
    
    ######################
    
    
    
    
    
        ####### CNBC ####### yesterday's all news -------> tekrar bak

    def scrape_cnbc_news_links(self):
        
        url = "https://www.cnbc.com/site-map/articles/yesterday/"
        
        soup = self.soup_maker(url)
        
        dom = etree.HTML(str(soup))

        links_element = dom.xpath('//div//div[contains(@class, "SiteMapArticleList-articleData")]/ul/li/a')

        new_rows_df = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])

        for link_ele in links_element:
            combine_whole_link = link_ele.get('href')
            
            # title, text, date = self.go_inside_cnbc_links(link = combine_whole_link)
            
            new_row = {"headline": "title", "alltext": "text", "date": "date", "source": "CNBC", "url": combine_whole_link}
            new_rows_df = new_rows_df.append(new_row, ignore_index=True)

        return new_rows_df
    
    
    def go_inside_cnbc_links(self, link):
        # for link in links: 
        soup = self.soup_maker(link)
        dom = etree.HTML(str(soup))

        title_element = dom.xpath('//div/div/header/h1[contains(@class, "text-3xl font-medium tracking-tight text-gray-1100 leading-relative-2 md:text-5xl")]/text()')
        if title_element != []:
            title = title_element[0]
        else:
            title = title_element
        # title = 'titlegelecek'
        
        text_element =dom.xpath('//div/div[contains(@class, "max-w-full")]/div[contains(@class, "article-body")]/p//text()')
        text = ''
        self.text_element = text_element
        for paragraph in text_element:
            text = text + paragraph
        # text= 'textgelecek'

        date_element = dom.xpath('//div/div[contains(@class, "text-lg font-medium text-gray-800 mt-12px md:mt-24px mb-24px")]/text()[2]')
        date = date_element
        try:
            date = date[0]
            date = date[3:-1]
        except:
            pass
        if 'Updated' in date:
            date = date.replace('Updated ', '')
                # orjinal format 'Jul 9, 2024 at 4:51AM'
        # date = pd.to_datetime(date, format='%b %d, %Y at %I:%M%p', errors='coerce')
        
        date = pd.to_datetime(date)
        
        # date = '13'


        return title, text, date
            
        
    
    
        
        
    def scrape_all_sites(self):
        
        df_data = pd.DataFrame(columns = ["headline", "alltext", "date", "source", "url"])
        df_data = df_data.append(self.scrape_fool_news_links(), ignore_index=True)
        df_data = df_data.append(self.scrape_benzinga_news_links(), ignore_index=True)
        df_data = df_data.append(self.scrape_yahoo_news_links(), ignore_index=True)
        # df_data = df_data.append(self.scrape_cnbc_news_links(), ignore_index=True)    # tekrar bakılmaı linkler çekilmiyor
        # df_data = df_data.append(self.scrape_barrons_news_links(), ignore_index=True)  # need subscription !!!!!!!!!!!!  captcha
        return df_data
        
        
if __name__ == '__main__':

    scraper = DataScraper()
    data = scraper.scrape_all_sites()
    data.to_excel("last_news.xlsx", index = False, engine = 'openpyxl')

    print(data)


                                              headline  \
0    Super Micro Computer Announces 10-for-1 Stock ...   
1          3 Dividend Stocks to Double Up on Right Now   
2    S&P 500 Sell-Off: Is It Really Safe to Invest ...   
3      Why Meta Platforms Stock Was Gaining on Tuesday   
4    I Used to Think I'd Claim Social Security at 7...   
..                                                 ...   
105  Trump’s Overtures to US Crypto Sector Put Glob...   
106      Prediction: Bitcoin Will Hit $100,000 by 2025   
107                        The 'Trump trade' is fading   
108  Stock market news today: Stocks rally to snap ...   
109  Bitcoin bulls predict new record highs followi...   

                                               alltext                 date  \
0    Recent advancements in the field of artificial...  2024-08-06 22:13:00   
1    With signs pointing to the Federal Reserve bei...  2024-08-06 11:45:00   
2    After enjoying record-breaking highs most of t...  2024-08-06

In [427]:
import datetime 
import pandas as pd
import pymongo
import warnings
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
from matplotlib.patches import FancyBboxPatch
import os
os.environ['CURL_CA_BUNDLE'] = ''
warnings.filterwarnings("ignore")
# client = pymongo.MongoClient('10.128.96.25', 27017)
# db = client["news"]

mydf33 = data

newsCollection = mydf33

newsCollection = newsCollection.reset_index(drop=True)
## delete empty rows

i = 0
while i <= len(newsCollection)-1:
    if newsCollection['headline'][i] == []:
        newsCollection = newsCollection.drop(i)
    i += 1
        

newsCollection = newsCollection.drop_duplicates()




sectors = [  'Diversified Telecommunication Services', 'Entertainment','Interactive Media & Services','Media',
             'Wireless Telecommunication Services','Show Consumer Discretionary details','Consumer Discretionary','Auto Components',
             'Automobiles','Distributors','Diversified Consumer Services','Hotels, Restaurants & Leisure','Household Durables',
             'Internet & Direct Marketing Retail','Leisure Products','Multiline Retail','Specialty Retail','Textiles, Apparel & Luxury Goods',
             'Consumer Staples','Beverages','Food & Staples Retailing','Food Products','Household Products','Personal Products','Tobacco',
             'Show Energy details','Energy','2 Industries','Energy Equipment & Services','Oil, Gas & Consumable Fuels','Financials','Banks',
             'Capital Markets','Consumer Finance','Diversified Financial Services','Insurance','Mortgage REITs','Thrifts & Mortgage Finance',
             'Show Health Care details','Health Care','6 Industries','Biotechnology','Health Care Equipment & Supplies',
             'Health Care Providers & Services','Health Care Technology','Life Sciences Tools & Services','Pharmaceuticals',
             'Industrials','Aerospace & Defense','Air Freight & Logistics','Airlines','Building Products','Commercial Services & Supplies',
             'Construction & Engineering','Electrical Equipment','Industrial Conglomerates','Machinery','Marine','Professional Services',
             'Road & Rail','Trading Companies & Distributors','Transportation Infrastructure','Information Technology','Communications Equipment',
             'Electronic Equipment, Instruments & Components','IT Services','Semiconductors','Semiconductor Equipment','Software',
             'Technology Hardware, Storage & Peripherals','Materials','Chemicals','Construction Materials','Containers & Packaging',
             'Metals & Mining','Paper & Forest Products','Real Estate','Equity Real Estate Investment Trusts',
             'Real Estate Management & Development','Show Utilities details','Utilities','Electric Utilities',
             'Gas Utilities','Independent Power and Renewable Electricity Producers','Multi-Utilities','Water Utilities']


others = [   "EMERGING MARKET", "GOLD", 
             'Nasdaq', 'S&P 500', "Dow Jones", "Russell 2000", "Brent Crude Oil", "Brent Petrol", "Gold Price" "Ons Gold",
             "FTSE", "Nikkei", "Vix", "Dax", "Dax 40", "Euro Stoxx", "NYSE", "Silver", "Treasury Yields", 
             "COVID", "covid","Trade War","energy shortage", "Energy Prices",
             "stagflasion", "Inflation", "recession"]
usTickers = ['3M', 'A. O. Smith','Abbott Laboratories','AbbVie','Accenture','Activision Blizzard',
             'ADM (company)','Adobe Inc.','ADP (company)','Advance Auto Parts','AES Corporation','Aflac','Agilent Technologies',
             'Air Products and Chemicals','Akamai','Alaska Air Group','Albemarle Corporation','Alexandria Real Estate Equities',
             'Align Technology','Allegion','Alliant Energy','Allstate','Alphabet Inc.','Alphabet Inc.','Altria','Amazon','Amcor',
             'Advanced Micro Devices','Ameren','American Airlines Group','American Electric Power','American Express','American International Group',
             'American Tower','American Water Works','Ameriprise Financial','AmerisourceBergen','Ametek','Amgen','Amphenol',
             'Analog Devices','Ansys','Aon (company)','APA Corporation','Apple Inc.','Applied Materials','Aptiv',
             'Arch Capital Group','Arista Networks','Arthur J. Gallagher & Co.','Assurant','AT&T','Atmos Energy',
             'Autodesk','AutoZone','AvalonBay Communities','Avery Dennison','Baker Hughes','Ball Corporation',
             'Bank of America','Bath & Body Works, Inc.','Baxter International','BD (company)','W. R. Berkley Corporation',
             'Berkshire Hathaway','Best Buy','Bio-Rad','Bio-Techne','Biogen','BlackRock','BNY Mellon','Boeing','Booking Holdings',
             'BorgWarner','Boston Properties','Boston Scientific','Bristol Myers Squibb','Broadcom Inc.','Broadridge Financial Solutions',
             'Brown & Brown','Brown–Forman','C.H. Robinson','Cadence Design Systems','Caesars Entertainment (2020)','Camden Property Trust',
             'Campbell Soup Company','Capital One','Cardinal Health','CarMax','Carnival Corporation & plc','Carrier Global','Catalent',
             'Caterpillar Inc.','Cboe Global Markets','CBRE Group','CDW','Celanese','Centene Corporation','CenterPoint Energy',
             'Ceridian','CF Industries','Charles River Laboratories','Charles Schwab Corporation','Charter Communications',
             'Chevron Corporation','Chipotle Mexican Grill','Chubb Limited','Church & Dwight','Cigna','Cincinnati Financial',
             'Cintas','Cisco','Citigroup','Citizens Financial Group','Clorox','CME Group','CMS Energy','The Coca-Cola Company',
             'Cognizant','Colgate-Palmolive','Comcast','Comerica','Conagra Brands','ConocoPhillips','Consolidated Edison',
             'Constellation Brands','Constellation Energy','CooperCompanies','Copart','Corning Inc.','Corteva','CoStar Group',
             'Costco','Coterra','Crown Castle','CSX Corporation','Cummins','CVS Health','D.R. Horton','Danaher Corporation',
             'Darden Restaurants','DaVita Inc.','John Deere','Delta Air Lines','Dentsply Sirona','Devon Energy','Dexcom',
             'Diamondback Energy','Digital Realty','Discover Financial','Dish Network','Disney','Dollar General',
             'Dollar Tree','Dominion Energy',"Domino's",'Dover Corporation','Dow Inc.','DTE Energy','Duke Energy',
             'DuPont','DXC Technology','Eastman Chemical Company','Eaton Corporation','EBay','Ecolab','Edison International',
             'Edwards Lifesciences','Electronic Arts','Elevance Health','Eli Lilly and Company','Emerson Electric','Enphase',
             'Entergy','EOG Resources','EPAM Systems','EQT','Equifax','Equinix','Equity Residential','Essex Property Trust',
             'The Estée Lauder Companies','Etsy','Everest Re','Evergy','Eversource','Exelon','Expedia Group','Expeditors International',
             'Extra Space Storage','ExxonMobil','F5, Inc.','FactSet','Fastenal','Federal Realty','FedEx','Fifth Third Bank',
             'First Republic Bank','First Solar','FirstEnergy','FIS (company)','Fiserv','Fleetcor','FMC Corporation',
             'Ford Motor Company','Fortinet','Fortive','Fox Corporation','Fox Corporation','Franklin Templeton',
             'Freeport-McMoRan','Garmin','Gartner','Gen Digital','Generac','General Dynamics','General Electric',
             'General Mills','General Motors','Genuine Parts Company','Gilead Sciences','Globe Life','Global Payments',
             'Goldman Sachs','Halliburton','The Hartford','Hasbro','HCA Healthcare','Healthpeak Properties','Henry Schein',
             'The Hershey Company','Hess Corporation','Hewlett Packard Enterprise','Hilton Worldwide','Hologic','The Home Depot',
             'Honeywell','Hormel Foods','Host Hotels & Resorts','Howmet Aerospace','HP Inc.','Humana','Huntington Bancshares',
             'Huntington Ingalls Industries','IBM','IDEX Corporation','Idexx Laboratories','Illinois Tool Works','Illumina, Inc.',
             'Incyte','Ingersoll Rand','Intel','Intercontinental Exchange','International Paper','The Interpublic Group of Companies',
             'International Flavors & Fragrances','Intuit','Intuitive Surgical','Invesco','Invitation Homes','IQVIA','Iron Mountain (company)',
             'J.B. Hunt','Jack Henry & Associates','Jacobs Solutions','Johnson & Johnson','Johnson Controls','JPMorgan Chase','Juniper Networks',
             "Kellogg's",'Keurig Dr Pepper','KeyBank','Keysight','Kimberly-Clark','Kimco Realty','Kinder Morgan','KLA Corporation',
             'Kraft Heinz','Kroger','L3Harris','LabCorp','Lam Research','Lamb Weston','Las Vegas Sands','Leidos','Lennar',
             'Lincoln Financial','Linde plc','Live Nation Entertainment','LKQ Corporation','Lockheed Martin','Loews Corporation',
             "Lowe's",'Lumen Technologies','LyondellBasell','M&T Bank','Marathon Oil','Marathon Petroleum','MarketAxess',
             'Marriott International','Marsh McLennan','Martin Marietta Materials','Masco','Mastercard','Match Group',
             'McCormick & Company',"McDonald's",'McKesson','Medtronic','Merck & Co.','Meta Platforms','MetLife','Mettler Toledo',
             'MGM Resorts','Microchip Technology','Micron Technology','Microsoft','Mid-America Apartment Communities','Moderna',
             'Mohawk Industries','Molina Healthcare','Molson Coors Beverage Company','Mondelez International','Monolithic Power Systems',
             'Monster Beverage',"Moody's Corporation",'Morgan Stanley','The Mosaic Company','Motorola Solutions','MSCI','Nasdaq, Inc.',
             'NetApp','Netflix','Newell Brands','Newmont','News Corp','News Corp','NextEra Energy','Nike, Inc.','NiSource',
             'Nordson Corporation','Norfolk Southern Railway','Northern Trust','Northrop Grumman','Norwegian Cruise Line Holdings',
             'NRG Energy','Nucor','Nvidia','NVR, Inc.','NXP Semiconductors',"O'Reilly Auto Parts",'Occidental Petroleum',
             'Old Dominion Freight Line','Omnicom Group','ON Semiconductor','Oneok','Oracle Corporation','Organon & Co.',
             'Otis Worldwide','Paccar','Packaging Corporation of America','Paramount Global','Parker Hannifin','Paychex',
             'Paycom','PayPal','Pentair','PepsiCo','PerkinElmer','Pfizer','PG&E','Philip Morris International','Phillips 66',
             'Pinnacle West','Pioneer Natural Resources','PNC Financial Services','Pool Corporation (page does not exist)',
             'PPG Industries','PPL Corporation','Principal Financial Group','Procter & Gamble','Progressive Corporation',
             'Prologis','Prudential Financial','Public Service Enterprise Group','PTC (software company)','Public Storage',
             'PulteGroup','Qorvo','Quanta Services','Qualcomm','Quest Diagnostics','Ralph Lauren Corporation','Raymond James',
             'Raytheon Technologies','Realty Income','Regency Centers','Regeneron','Regions Financial Corporation','Republic Services',
             'ResMed','Robert Half','Rockwell Automation','Rollins, Inc.','Roper Technologies','Ross Stores','Royal Caribbean Group',
             'S&P Global','Salesforce','SBA Communications','Schlumberger','Seagate Technology','Sealed Air','Sempra Energy',
             'ServiceNow','Sherwin-Williams','Signature Bank','Simon Property Group','Skyworks Solutions',
             'The J.M. Smucker Company','Snap-on','SolarEdge','Southern Company','Southwest Airlines','Stanley Black & Decker',
             'Starbucks','State Street Corporation','Steel Dynamics','Steris','Stryker Corporation','Silicon Valley Bank',
             'Synchrony Financial','Synopsys','Sysco','T-Mobile US','T. Rowe Price','Take-Two Interactive','Tapestry, Inc.',
             'Targa Resources','Target Corporation','TE Connectivity','Teledyne Technologies','Teleflex','Teradyne','Tesla, Inc.',
             'Texas Instruments','Textron','Thermo Fisher Scientific','TJX Companies','Tractor Supply','Trane Technologies',
             'TransDigm Group','The Travelers Companies','Trimble Inc.','Truist Financial','Tyler Technologies','Tyson Foods',
             'U.S. Bank','UDR, Inc.','Ulta Beauty','Union Pacific Corporation','United Airlines Holdings','United Parcel Service',
             'United Rentals','UnitedHealth Group','Universal Health Services','Valero Energy','Ventas (company)','Verisign',
             'Verisk','Verizon','Vertex Pharmaceuticals','VF Corporation','Viatris','Vici Properties','Visa Inc.',
             'Vornado Realty Trust','Vulcan Materials Company','Wabtec','Walgreens Boots Alliance','Walmart',
             'Warner Bros. Discovery','Waste Management (corporation)','Waters Corporation','WEC Energy Group',
             'Wells Fargo','Welltower','West Pharmaceutical Services','Western Digital','WestRock','Weyerhaeuser',
             'Whirlpool Corporation','Williams Companies','Willis Towers Watson','W. W. Grainger','Wynn Resorts',
             'Xcel Energy','Xylem Inc.','Yum! Brands','Zebra Technologies','Zimmer Biomet','Zions Bancorporation','Zoetis']
usTickers = usTickers + sectors + others

usTickers = [each.upper() for each in usTickers]

torch.set_num_threads(6) 
def calculateSentiment(text):
    inputs = tokenizer([text], padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    model.config.id2label
    #Tweet #Positive #Negative #Neutral
    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()
    
    table = {
             "Positive":positive,
             "Negative":negative, 
             "Neutral":neutral}
    return positive[0], negative[0], neutral[0]

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    sentences = [s.upper() for s in sentences]
    return sentences
proxies = {
    "https" : "fnyproxy.fnylocal.com:8080",
    "http" : "fnyproxy.fnylocal.com:8080"
}


newsCollection.reset_index(inplace = True, drop = True)
#a["date"] = a["date"].apply(lambda x:  x[:10] if type(x) != type(datetime.datetime(2022,11,11)) else x)
#a["date"] = a["date"].apply(lambda x:  datetime.datetime.strptime(x, '%Y-%m-%d') if type(x) != type(datetime.datetime(2022,11,11)) else x)
# a["date"] = a["date"].apply(lambda x: x.replace(hour = 0, minute = 0, second = 0, microsecond = 0))
i = 0
for d in newsCollection["date"]:
    newsCollection["date"][i] = str(d)[:10]
    i += 1
print(newsCollection["date"])

#a = a[a["date"] >= datetime.datetime(2023,8,19)]
# a = a.reset_index().drop("index", axis = 1)
a = newsCollection

array = np.array(a.headline)
print(len(a))

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert", proxies = proxies)
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", proxies = proxies)
def historicalSentiment(df):
    tempTickers = []
    tempPosList = []
    tempNegList = []
    tempNeuList = []
    sentence_count = 0
    ticker_count = 0

    for index, row in df.iterrows():
        sentenceList = split_into_sentences(row['alltext'])
        for sentence in sentenceList:
            sentence_count += 1 
            for ticker in usTickers:
                if ticker in sentence:
                    ticker_count += 1 
                    tempTickers.append(ticker)
                    sentiment = calculateSentiment(sentence)
                    tempPosList.append(sentiment[0])
                    tempNegList.append(sentiment[1])
                    tempNeuList.append(sentiment[2])
    
    if tempTickers:
        resultDf = pd.DataFrame(list(zip(tempTickers, tempPosList, tempNegList, tempNeuList)), columns=["ticker", "positive", "negative", "neutral"])
        resultDf["positive"] = resultDf["positive"] * 100
        resultDf["negative"] = resultDf["negative"] * 100
        resultDf["neutral"] = resultDf["neutral"] * 100
        resultDf["date"] = df['date'].iloc[-1]
        resultDf = resultDf.groupby(["ticker"]).mean().reset_index()
    else:
        resultDf = pd.DataFrame()

    return resultDf





0     2024-08-06
1     2024-08-06
2     2024-08-06
3     2024-08-07
4     2024-08-07
         ...    
61    2024-08-07
62    2024-08-07
63    2024-08-07
64    2024-08-07
65    2024-08-06
Name: date, Length: 66, dtype: object
66


In [ ]:

# Ensure columns exist
if 'tickers' not in a.columns:
    a['tickers'] = np.nan
if 'positive' not in a.columns:
    a['positive'] = np.nan
if 'negative' not in a.columns:
    a['negative'] = np.nan
if 'neutral' not in a.columns:
    a['neutral'] = np.nan

# Ensure each column is ready to accept list type
a['tickers'] = a['tickers'].astype(object)
a['positive'] = a['positive'].astype(object)
a['negative'] = a['negative'].astype(object)
a['neutral'] = a['neutral'].astype(object)

# Define the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert", proxies=proxies)
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", proxies=proxies)

# Update historicalSentiment function if needed

# Iterate over DataFrame safely
for index, row in a.iterrows():
    # Single row DataFrame
    single_news_df = a.loc[[index]]

    # Analyze sentiment
    sentiment_df = historicalSentiment(single_news_df)

    # If sentiment_df is not empty, store results
    if not sentiment_df.empty:
        a.at[index, 'tickers'] = sentiment_df['ticker'].tolist()
        a.at[index, 'positive'] = sentiment_df['positive'].tolist()
        a.at[index, 'negative'] = sentiment_df['negative'].tolist()
        a.at[index, 'neutral'] = sentiment_df['neutral'].tolist()
    else:
        a.at[index, 'tickers'] = []
        a.at[index, 'positive'] = []
        a.at[index, 'negative'] = []
        a.at[index, 'neutral'] = []

# Check results
# print(a[['date', 'tickers', 'positive', 'negative', 'neutral']])

In [ ]:
a = a.drop_duplicates(subset = ['headline'])
a['url'][0]

In [ ]:
a['status'] = np.nan
a['status'] = a['status'].astype(object)

In [ ]:
a = a.reset_index(drop=True)

In [ ]:
for row_index in range(0,len(a)):
    status = []
    for ticker_index in range(0,len(a['tickers'][row_index])):
        sentiment = {'positive' : a['positive'][row_index][ticker_index], 'negative' : a['negative'][row_index][ticker_index], 'neutral' : a['neutral'][row_index][ticker_index]}
        compare = [(value, key) for key, value in sentiment.items()]
        status.append(max(compare)[1])
    a['status'][row_index] = status

In [ ]:
status

In [ ]:
a

In [ ]:
client = pymongo.MongoClient('10.128.96.25', 27017)
db = client["sentiment_news"]
sentiment_newsCollection = db["sentiment_newsCollection"]
sentiment_news_df = pd.DataFrame(list(sentiment_newsCollection.find()))

sentiment_news_df = sentiment_news_df.drop('_id', axis=1)
db.sentiment_newsCollection.drop()

b = pd.concat([sentiment_news_df, a])
b = b.drop_duplicates(subset=['alltext', 'headline'], ignore_index=True)
# print(sum(b.duplicated(subset=['headline'])))
b

In [ ]:
# import pymongo

Connection_String = "mongodb://mongohost:27017"
session = pymongo.MongoClient('10.128.96.25', 27017)
db = session['sentiment_news']
collection = db['sentiment_newsCollection']

data_records_b = b.to_dict("records")
collection.insert_many(data_records_b)

In [ ]:
b

In [ ]:
b[b['date'] == '2024-08-05']